In [11]:
# importing packages for the project

In [12]:
import pandas as pd
import numpy as np
!pip install beautifulsoup4
import requests
from bs4 import BeautifulSoup

In [13]:
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

In [14]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

# All requested packages already installed.



In [15]:
import folium

In [16]:
!conda install -c conda-forge geopy --yes

Solving environment: done

# All requested packages already installed.



In [17]:
from geopy.geocoders import Nominatim

In [18]:
# getting neighbourhood data

In [45]:
# wikipedia site with postal codes and neighbourhoods
wiki_url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# scraping data
req = requests.get(wiki_url) 
soup = BeautifulSoup(req.content,'lxml') 
table = soup.find_all('table')[0]  
df = pd.read_html(str(table)) 
neighbourhood=pd.DataFrame(df[0])
# putting data into dataframe and cleaning
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighbourhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [46]:
df.head(5)

AttributeError: 'NoneType' object has no attribute 'items'

  PostalCode           Borough                     Neighbourhood
0        M3A        North York                         Parkwoods
1        M4A        North York                  Victoria Village
2        M5A  Downtown Toronto         Regent Park, Harbourfront
3        M6A        North York  Lawrence Manor, Lawrence Heights
4        M7A      Queen's Park     Ontario Provincial Government

In [47]:
# loading csv file with coordinates by postal code and matching column name
load = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
coord = load.rename(columns = {"Postal Code":"PostalCode"})
# checking coord loaded properly
coord.head(5)

AttributeError: 'NoneType' object has no attribute 'items'

  PostalCode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711
3        M1G  43.770992 -79.216917
4        M1H  43.773136 -79.239476

In [48]:
# joining coord to df by postal codes
full_df = df.set_index('PostalCode').join(coord.set_index('PostalCode'))
# checking full_df
full_df.head(5)

AttributeError: 'NoneType' object has no attribute 'items'

                     Borough                     Neighbourhood   Latitude  \
PostalCode                                                                  
M3A               North York                         Parkwoods  43.753259   
M4A               North York                  Victoria Village  43.725882   
M5A         Downtown Toronto         Regent Park, Harbourfront  43.654260   
M6A               North York  Lawrence Manor, Lawrence Heights  43.718518   
M7A             Queen's Park     Ontario Provincial Government  43.662301   

            Longitude  
PostalCode             
M3A        -79.329656  
M4A        -79.315572  
M5A        -79.360636  
M6A        -79.464763  
M7A        -79.389494  

In [49]:
# creating df for each desired Toronto borough
ET = full_df[full_df['Borough'] == 'East Toronto'].reset_index(drop=True)
DT = full_df[full_df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
WT = full_df[full_df['Borough'] == 'West Toronto'].reset_index(drop=True)
NT = full_df[full_df['Borough'] == 'North Toronto'].reset_index(drop=True)

In [50]:
# combining all desired Toronto boroughs into one dataframe
toronto_data = ET.append(DT, ignore_index=True)
toronto_data = toronto_data.append(WT, ignore_index=True)
toronto_data = toronto_data.append(NT, ignore_index=True)

In [51]:
# retrieving coordinates of Toronto for mapping purposes
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordinates {}, {}.'.format(latitude, longitude))

Coordinates 43.6534817, -79.3839347.


In [52]:
# Foursquare API request for all venues in neighbourhoods

In [53]:
# API credentials

In [54]:
# The code was removed by Watson Studio for sharing.

In [55]:
# version and limit for API request
VERSION = '20180605'
LIMIT = 100

In [56]:
# API request to get venues in Toronto boroughs
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [57]:
# venues in all Toronto boroughs
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction South
Parkdale, Roncesvalles
Runnymede, Swansea


In [58]:
# checking numbers of each venue category
toronto_venues['Venue Category'].value_counts()

Coffee Shop      121
Café              75
Restaurant        46
Bakery            39
Hotel             36
                ... 
BBQ Joint          1
Soup Place         1
College Gym        1
Historic Site      1
Butcher            1
Name: Venue Category, Length: 220, dtype: int64

In [59]:
# make dfs for each organic-type category
organic_grocery = toronto_venues[toronto_venues['Venue Category'] == 'Organic Grocery'].reset_index(drop=True)
health_food = toronto_venues[toronto_venues['Venue Category'] == 'Health Food Store'].reset_index(drop=True)
farmers_market = toronto_venues[toronto_venues['Venue Category'] == 'Farmers Market'].reset_index(drop=True)
fruit_vegetable = toronto_venues[toronto_venues['Venue Category'] == 'Fruit & Vegetable Store'].reset_index(drop=True)
# append dfs into one df
organic = organic_grocery.append(health_food, ignore_index=True)
organic = organic.append(farmers_market, ignore_index=True)
organic

AttributeError: 'NoneType' object has no attribute 'items'

                                Neighbourhood  Neighbourhood Latitude  \
0   Kensington Market, Chinatown, Grange Park               43.653206   
1                                 The Beaches               43.676357   
2                   Regent Park, Harbourfront               43.654260   
3                          Runnymede, Swansea               43.651571   
4                   Regent Park, Harbourfront               43.654260   
5                              St. James Town               43.651494   
6                              St. James Town               43.651494   
7                                 Berczy Park               43.644771   
8                                 Berczy Park               43.644771   
9   Kensington Market, Chinatown, Grange Park               43.653206   
10                St. James Town, Cabbagetown               43.667967   

    Neighbourhood Longitude                                 Venue  \
0                -79.400049              Essence of Li

In [60]:
# make dfs for each conventional-type category
grocery_store = toronto_venues[toronto_venues['Venue Category'] == 'Grocery Store'].reset_index(drop=True)
supermarket = toronto_venues[toronto_venues['Venue Category'] == 'Supermarket'].reset_index(drop=True)
# append dfs into one df
conventional = grocery_store.append(supermarket, ignore_index=True)

In [82]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)
# add markers to map for conventional
for lat, lng, label in zip(conventional['Venue Latitude'], conventional['Venue Longitude'], conventional['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=False,
        parse_html=False).add_to(map_toronto)  
# add markers to map for organic
for lat, lng, label in zip(organic['Venue Latitude'], organic['Venue Longitude'], organic['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=False,
        parse_html=False).add_to(map_toronto) 
map_toronto

In [62]:
# count of conventional by neighbourhood
conventionaltally = conventional.groupby('Neighbourhood').count()
conventionaltally = conventionaltally.drop(['Neighbourhood Latitude','Neighbourhood Longitude','Venue Latitude','Venue Longitude','Venue Category'], axis=1)
conventionaltally = conventionaltally.rename(columns = {"Venue":"Conventional"})
conventionaltally

AttributeError: 'NoneType' object has no attribute 'items'

                                                   Conventional
Neighbourhood                                                  
Berczy Park                                                   1
Brockton, Parkdale Village, Exhibition Place                  1
Christie                                                      4
Church and Wellesley                                          1
Dufferin, Dovercourt Village                                  2
Harbourfront East, Union Station, Toronto Islands             1
High Park, The Junction South                                 1
Kensington Market, Chinatown, Grange Park                     2
St. James Town                                                1
St. James Town, Cabbagetown                                   1
The Danforth West, Riverdale                                  1

In [63]:
# count of organic by neighbourhood
organictally = organic.groupby('Neighbourhood').count()
organictally = organictally.drop(['Neighbourhood Latitude','Neighbourhood Longitude','Venue Latitude','Venue Longitude','Venue Category'], axis=1)
organictally = organictally.rename(columns = {"Venue":"Organic"})
organictally

AttributeError: 'NoneType' object has no attribute 'items'

                                           Organic
Neighbourhood                                     
Berczy Park                                      2
Kensington Market, Chinatown, Grange Park        2
Regent Park, Harbourfront                        2
Runnymede, Swansea                               1
St. James Town                                   2
St. James Town, Cabbagetown                      1
The Beaches                                      1

In [64]:
# df comparing count of organic and conventional in each neighbourhood
tally = pd.merge(organictally, conventionaltally, how='outer', on='Neighbourhood')
tally

AttributeError: 'NoneType' object has no attribute 'items'

                                                   Organic  Conventional
Neighbourhood                                                           
Berczy Park                                            2.0           1.0
Kensington Market, Chinatown, Grange Park              2.0           2.0
Regent Park, Harbourfront                              2.0           NaN
Runnymede, Swansea                                     1.0           NaN
St. James Town                                         2.0           1.0
St. James Town, Cabbagetown                            1.0           1.0
The Beaches                                            1.0           NaN
Brockton, Parkdale Village, Exhibition Place           NaN           1.0
Christie                                               NaN           4.0
Church and Wellesley                                   NaN           1.0
Dufferin, Dovercourt Village                           NaN           2.0
Harbourfront East, Union Station, Toronto Islands  

In [78]:
# adding a calulated column for the ratio of organic to conventional stores in the neighbourhood, made ratio for undefined (division by 0) 1000
tally['Ratio'] = [2.0,1.0,1000,1000,2.0,1.0,1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
tally

AttributeError: 'NoneType' object has no attribute 'items'

                                                   Organic  Conventional  \
Neighbourhood                                                              
Berczy Park                                            2.0           1.0   
Kensington Market, Chinatown, Grange Park              2.0           2.0   
Regent Park, Harbourfront                              2.0           NaN   
Runnymede, Swansea                                     1.0           NaN   
St. James Town                                         2.0           1.0   
St. James Town, Cabbagetown                            1.0           1.0   
The Beaches                                            1.0           NaN   
Brockton, Parkdale Village, Exhibition Place           NaN           1.0   
Christie                                               NaN           4.0   
Church and Wellesley                                   NaN           1.0   
Dufferin, Dovercourt Village                           NaN           2.0   
Harbourfront

In [79]:
# df with just the neighbourhood and ratio for plotting
ratio = tally.drop(['Organic','Conventional'], axis=1)
ratio

AttributeError: 'NoneType' object has no attribute 'items'

                                                    Ratio
Neighbourhood                                            
Berczy Park                                           2.0
Kensington Market, Chinatown, Grange Park             1.0
Regent Park, Harbourfront                          1000.0
Runnymede, Swansea                                 1000.0
St. James Town                                        2.0
St. James Town, Cabbagetown                           1.0
The Beaches                                        1000.0
Brockton, Parkdale Village, Exhibition Place          0.0
Christie                                              0.0
Church and Wellesley                                  0.0
Dufferin, Dovercourt Village                          0.0
Harbourfront East, Union Station, Toronto Islands     0.0
High Park, The Junction South                         0.0
The Danforth West, Riverdale                          0.0

In [80]:
# adding neighbourhood coordinates
ratio = pd.merge(ratio, toronto_data, how='inner', on='Neighbourhood')
ratio = ratio.drop(['Borough'], axis=1)
ratio

AttributeError: 'NoneType' object has no attribute 'items'

                                        Neighbourhood   Ratio   Latitude  \
0                                         Berczy Park     2.0  43.644771   
1           Kensington Market, Chinatown, Grange Park     1.0  43.653206   
2                           Regent Park, Harbourfront  1000.0  43.654260   
3                                  Runnymede, Swansea  1000.0  43.651571   
4                                      St. James Town     2.0  43.651494   
5                         St. James Town, Cabbagetown     1.0  43.667967   
6                                         The Beaches  1000.0  43.676357   
7        Brockton, Parkdale Village, Exhibition Place     0.0  43.636847   
8                                            Christie     0.0  43.669542   
9                                Church and Wellesley     0.0  43.665860   
10                       Dufferin, Dovercourt Village     0.0  43.669005   
11  Harbourfront East, Union Station, Toronto Islands     0.0  43.640816   
12          

In [81]:
# making new dataframe for the neighbourhoods with a ratio of zero (have conventional but no organic store)
neworganic = ratio.iloc[[8,9,10,11,12,13],:]
neworganic

AttributeError: 'NoneType' object has no attribute 'items'

                                        Neighbourhood  Ratio   Latitude  \
8                                            Christie    0.0  43.669542   
9                                Church and Wellesley    0.0  43.665860   
10                       Dufferin, Dovercourt Village    0.0  43.669005   
11  Harbourfront East, Union Station, Toronto Islands    0.0  43.640816   
12                      High Park, The Junction South    0.0  43.661608   
13                       The Danforth West, Riverdale    0.0  43.679557   

    Longitude  
8  -79.422564  
9  -79.383160  
10 -79.442259  
11 -79.381752  
12 -79.464763  
13 -79.352188  

In [83]:
# mapping current grocery store locations by type and neighbourhoods suitable for new organic stores
# create map of Toronto using latitude and longitude values
map_final = folium.Map(location=[latitude, longitude], zoom_start=13)
# add markers to map for conventional
for lat, lng, label in zip(conventional['Venue Latitude'], conventional['Venue Longitude'], conventional['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=False,
        parse_html=False).add_to(map_final)  
# add markers to map for organic
for lat, lng, label in zip(organic['Venue Latitude'], organic['Venue Longitude'], organic['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=False,
        parse_html=False).add_to(map_final) 
# add markers to map for new organic neighbourhoods
for lat, lng, label in zip(neworganic['Latitude'], neworganic['Longitude'], neworganic['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=False,
        parse_html=False).add_to(map_final)
map_final